In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np

In [3]:
import pandas as pd

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1sCb1YbQ3-1oiL-cnK0yCkwFcQpvpf0efIngAUeC1ixo"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### Remote download needs:
#### Natality by year
#### Natality by year and race
#### Natality by year and state
#### Grouped or filtered by preventable risk factors
#### Add in violent causes of death

In [9]:
natality_year = pd.read_table(
    "data/raw/natality_byyear.txt", dtype={"Year": str, "Births": int}, skipfooter=22
).drop(columns=["Notes", "Year Code"])

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_76491/3213058221.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  natality_year = pd.read_table(


### Wonder request needs:
#### all maternal mortality causes by cause and year
#### all maternal mortality causes by cause and year and race
#### all maternal mortality causes by cause and year and state
#### id 'preventable' maternal mortality?

### Looking at annual rates

In [10]:
df = pd.read_table("data/raw/byyear_andcause.txt", dtype={"Year": str}).drop(
    columns=["Year Code"]
)
df = df[~df["Deaths"].isna()]

In [11]:
df

,Notes,Year,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
0,NaN,1999,Tubal pregnancy,O00.1,5.0,2.790402e+08,Unreliable
1,NaN,1999,"Ectopic pregnancy, unspecified",O00.9,14.0,2.790402e+08,Unreliable
2,NaN,1999,"Hydatidiform mole, unspecified",O01.9,1.0,2.790402e+08,Unreliable
3,NaN,1999,Blighted ovum and nonhydatidiform mole,O02.0,1.0,2.790402e+08,Unreliable
4,NaN,1999,"Spontaneous abortion, complete or unspecified,...",O03.5,1.0,2.790402e+08,Unreliable
...,...,...,...,...,...,...,...
1708,NaN,2020,Diseases of the respiratory system complicatin...,O99.5,23.0,3.294841e+08,0.0
1709,NaN,2020,Diseases of the digestive system complicating ...,O99.6,2.0,3.294841e+08,Unreliable
1710,NaN,2020,Other specified diseases and conditions compli...,O99.8,94.0,3.294841e+08,0.0
1711,Total,2020,NaN,NaN,861.0,3.294841e+08,0.3


In [12]:
rates_perlivebirths = df.merge(natality_year, on="Year")
rates_perlivebirths["rate"] = 100000 * (
    rates_perlivebirths["Deaths"] / rates_perlivebirths["Births"]
)

In [13]:
rates_perlivebirths.columns = rates_perlivebirths.columns.str.lower().str.replace(
    " ", "_"
)

In [14]:
pd.set_option("display.max_columns", None)

top_five = (
    rates_perlivebirths[~rates_perlivebirths["cause_of_death_code"].isna()]
    .sort_values(["year", "rate"], ascending=False)
    .groupby("year")
    .head(5)
    .drop(columns=["notes", "population"])
)

In [15]:
wide_top5 = top_five.pivot(
    index="cause_of_death", columns="year", values="rate"
).fillna(0)

### top five causes over time

In [16]:
alt.Chart(top_five).mark_line(point=True).encode(
    x=alt.X("year:O"), y=alt.Y("rate"), color="cause_of_death:N"
)

alt.Chart(...)

### By year and race
#### start with deaths
#### prefiltered for mm causes (including timing)
#### then combine race and hispanic origin

In [17]:
df = pd.read_table("data/raw/byyear_cause_race.txt", dtype={"Year": str}).drop(
    columns=["Year Code"]
)
df = df[~df["Deaths"].isna()]
df["Year"] = df["Year"].str.strip()

In [18]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [19]:
race_ho_conditions = [
    ((df["race"] == "White") & (df["hispanic_origin"] == "Not Hispanic or Latino")),
    (
        (df["race"] == "Black or African American")
        & (df["hispanic_origin"] == "Not Hispanic or Latino")
    ),
    (
        (df["race"] == "Asian or Pacific Islander")
        & (df["hispanic_origin"] == "Not Hispanic or Latino")
    ),
    (
        (df["race"] == "American Indian or Alaska Native")
        & (df["hispanic_origin"] == "Not Hispanic or Latino")
    ),
    (df["hispanic_origin"] == "Hispanic or Latino")
    & (df["notes"] != "Total")
    & ~df["race"].isna(),
    (df["race"].isna()) & (df["hispanic_origin"].isna()) & (df["notes"] != "Total"),
]

race_ho_values = [
    "White",
    "Black",
    "AAPI",
    "American Indian or Alaska Native",
    "Hispanic or Latino",
    "Overall",
]

df["race_whispanicorigin"] = np.select(race_ho_conditions, race_ho_values)

### Natality
#### only single race available for 2020
#### so have to process that separately than 2007-2019 (which has bridged race categories)
#### then combine

##### 2020 births by race and year

In [20]:
natality_2020_race = pd.read_table(
    "data/raw/natality_2020_race.txt", dtype={"Year": str}
).drop(
    columns=["Year Code", "Mother's Hispanic Origin Code", "Mother's Single Race Code"]
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/natality_2020_race.txt'

In [ ]:
natality_2020_race = natality_2020_race[~natality_2020_race["Births"].isna()]

In [ ]:
natality_2020_race.columns = (
    natality_2020_race.columns.str.lower().str.replace(" ", "_").str.replace("'", "")
)

In [ ]:
aapi = ["Native Hawaiian or Other Pacific Islander", "Asian"]

birth_conditions = [
    (
        (natality_2020_race["mothers_single_race"] == "White")
        & (natality_2020_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_2020_race["notes"] != "Total")
    ),
    (
        (natality_2020_race["mothers_single_race"] == "Black or African American")
        & (natality_2020_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_2020_race["notes"] != "Total")
    ),
    (
        (natality_2020_race["mothers_single_race"].isin(aapi))
        & (natality_2020_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_2020_race["notes"] != "Total")
    ),
    (
        (
            natality_2020_race["mothers_single_race"]
            == "American Indian or Alaska Native"
        )
        & (natality_2020_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_2020_race["notes"] != "Total")
    ),
    (
        (natality_2020_race["mothers_hispanic_origin"] == "Hispanic or Latino")
        & (natality_2020_race["notes"] != "Total")
    ),
]

birth_values = [
    "White",
    "Black",
    "AAPI",
    "American Indian or Alaska Native",
    "Hispanic or Latino",
]

natality_2020_race["race_whispanicorigin"] = np.select(birth_conditions, birth_values)

In [ ]:
birth_2020_df = (
    natality_2020_race[natality_2020_race["race_whispanicorigin"] != "0"]
    .groupby(["year", "race_whispanicorigin"])
    .agg(total_births=pd.NamedAgg(column="births", aggfunc=sum))
    .reset_index()
)

#### Now 2007-2019

In [ ]:
natality_year_race = pd.read_table(
    "data/raw/natality_byyear_race_0719.txt", dtype={"Year": str}
).drop(
    columns=["Year Code", "Mother's Hispanic Origin Code", "Mother's Bridged Race Code"]
)

In [ ]:
natality_year_race = natality_year_race[~natality_year_race["Births"].isna()]

In [ ]:
natality_year_race.columns = (
    natality_year_race.columns.str.lower().str.replace(" ", "_").str.replace("'", "")
)

In [ ]:
birth_conditions = [
    (
        (natality_year_race["mothers_bridged_race"] == "White")
        & (natality_year_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_year_race["notes"] != "Total")
    ),
    (
        (natality_year_race["mothers_bridged_race"] == "Black or African American")
        & (natality_year_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_year_race["notes"] != "Total")
    ),
    (
        (natality_year_race["mothers_bridged_race"] == "Asian or Pacific Islander")
        & (natality_year_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_year_race["notes"] != "Total")
    ),
    (
        (
            natality_year_race["mothers_bridged_race"]
            == "American Indian or Alaska Native"
        )
        & (natality_year_race["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_year_race["notes"] != "Total")
    ),
    (
        (natality_year_race["mothers_hispanic_origin"] == "Hispanic or Latino")
        & (natality_year_race["notes"] != "Total")
    ),
]

birth_values = [
    "White",
    "Black",
    "AAPI",
    "American Indian or Alaska Native",
    "Hispanic or Latino",
]

natality_year_race["race_whispanicorigin"] = np.select(birth_conditions, birth_values)

In [ ]:
birth_year_df = (
    natality_year_race[natality_year_race["race_whispanicorigin"] != "0"]
    .groupby(["year", "race_whispanicorigin"])
    .agg(total_births=pd.NamedAgg(column="births", aggfunc=sum))
    .reset_index()
)

In [ ]:
births_year_race_0720 = pd.concat([birth_2020_df, birth_year_df])

In [ ]:
rates_cause_year_race = df.merge(
    births_year_race_0720, on=["year", "race_whispanicorigin"]
)

In [ ]:
rates_cause_year_race["rate"] = 100000 * (
    rates_cause_year_race["deaths"] / rates_cause_year_race["total_births"]
)

In [ ]:
top5_byrace = (
    rates_cause_year_race[~rates_cause_year_race["cause_of_death_code"].isna()]
    .sort_values(["year", "rate"], ascending=False)
    .groupby(["year", "race_whispanicorigin"])
    .head(5)
    .drop(
        columns=[
            "notes",
            "population",
            "race",
            "race_code",
            "hispanic_origin",
            "hispanic_origin_code",
            "crude_rate",
            "total_births",
            "deaths",
        ]
    )
)

In [ ]:
base_chart = (
    alt.Chart(top5_byrace)
    .mark_line(point=True)
    .encode(x=alt.X("year:O"), y=alt.Y("rate"), color="cause_of_death:N")
    .properties(width=100, height=200)
)

alt.layer(base_chart, data=top5_byrace).facet("race_whispanicorigin:N")

##### quality check rates to make sure they are correct
##### calculate mm rates for racial group by 2020 to make sure they line up with cdc report

In [ ]:
excluded_causes = ["O96", "O96.0", "O96.1", "O96.9", "O97", "O97.0", "O97.1", "O97.9"]

In [ ]:
test = rates_cause_year_race[
    (rates_cause_year_race["year"] == "2020")
    & (~rates_cause_year_race["cause_of_death_code"].isin(excluded_causes))
]

In [ ]:
test = test.drop(
    columns=[
        "notes",
        "race",
        "race_code",
        "hispanic_origin",
        "hispanic_origin_code",
        "population",
        "crude_rate",
        "rate",
    ]
)

In [ ]:
test = (
    test.groupby(["race_whispanicorigin", "total_births"])
    .agg(deaths_test=pd.NamedAgg(column="deaths", aggfunc=sum))
    .reset_index()
)

In [ ]:
test["rate_race"] = 100000 * (test["deaths_test"] / test["total_births"])

In [ ]:
test

In [ ]:
hispanic_latino = df[
    (df["race_whispanicorigin"] == "Hispanic or Latino") & (df["year"] == "2020")
]

In [ ]:
hispanic_latino.to_csv("data/processed/hispanic_latino_duplicated.csv")

##### black and white rates seemed congruent (w/in rounding errors) with CDC findings--Black: 55.3 per 100k live births, white: 19.1
##### seem to be double counting hispanic/latino (actual 2020 rate is 18)

#### What I'm seeing in the charts is that deaths that are excluded from CDC measures due to timing are actually often among the top causes in a given year for certain racial groups. are these racial groups minorities? does excluding these causes obscure maternal death inequality?

##### i have a datawrapper chart that compared this and had the correct rate for hispanic/latino, but when i repeated the methodology it didn't work? 
##### also try different years